## Linear classifier in pure Tensorflow

In this example, we are classifying samples based on their values (positive or negative). We start by randomly generating two sample sets, then combining them into a rank-2 tensor (matrix). We then define and run the training loop, and plot our predictions.

The algorithm for the training loop is as follows:
1. Drawing a batch of training samples with their corresponding targets
2. Running the model to obtain predictions (forward pass)
3. Compute the loss of the model on the batch
4. Compute the gradient of the loss with regards to the model's parameters
5. Moving the parameters in the opposite direction from the gradient, reducing the loss on the batch.
6. Repeat until we are satisfied with the loss

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


# Generating two classes of random points in a 2D plane
num_samples_per_class = 1000 

negative_samples = np.random.multivariate_normal(
    mean=[0, 3],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)

positive_samples = np.random.multivariate_normal(
    mean=[3, 0],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)

# Stacking the two classes into an array with shape (2000, 2)
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)


# Generating the corresponding targets (0 and 1)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))


# Plotting the two point classes
plt.scatter(inputs[:,0], inputs[:,1], c=targets[:,0])
plt.show()


# Creating linear classification variables
input_dim = 2   # number of features in input data
output_dim = 1  # number of neurons in the next layer of the network

# Weight matrix
W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim)))
# Bias vector - added to the output of the weight matrix
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,)))


# Defining the "forward pass" function
def model(inputs):
  return tf.matmul(inputs, W) + b


# Defining our "loss" function
def square_loss(targets, predictions):
  per_sample_losses = tf.square(targets - predictions)
  return tf.reduce_mean(per_sample_losses)


# Defining the "training step" function
learning_rate = 0.1

def training_step(inputs, targets):
  """
    Receives some training data and updates weights 
    W and b to minimize the loss on the data
  """
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = square_loss(targets, predictions)
  grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W,b]) 
  W.assign_sub(grad_loss_wrt_W * learning_rate)
  b.assign_sub(grad_loss_wrt_b * learning_rate) 

  return loss


# Batch training loop - runs each training step for all the data
for step in range(40):
  loss = training_step(inputs, targets)
  print(f"Loss at step {step}: {loss:.4f}")


# Plot our predictions
predictions = model(inputs)
#      x-coordinates   y-coordinates   set color of each point
plt.scatter(inputs[:,0], inputs[:,1], c=predictions[:,0] > 0.5)
plt.show()


# Plot a line to separate the two classes
x = np.linspace(-1,4,100)
y = - W[0] / W[1] * x + (0.5 - b) / W[1]    # y = mx + b
plt.plot(x, y, "-r")
plt.scatter(inputs[:,0], inputs[:,1], c=predictions[:,0] > 0.5)

